In [ ]:
import os
! cd .. && python -m pip install .

In [ ]:
import argparse
import wave

from ondewo.s2t import speech_to_text_pb2
from ondewo.s2t.client.client import Client
from ondewo.s2t.client.client_config import ClientConfig
from ondewo.s2t.client.services.speech_to_text import Speech2Text
from ondewo.s2t.speech_to_text_pb2 import ListS2tPipelinesRequest, Speech2TextConfig

## Creating a client object
The example below shows how to create a speech to text client object from a client config. 
When setting *use_secure_channel=True*, a grpc certificate *grpc_cert* is required.

In [ ]:
AUDIO_FILE: str = "examples/audiofiles/sample_1.wav"
# credentials = grpc.ssl_channel_credentials(root_certificates=cert)

MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "<ADD GRPC SERVER HERE>"
GRPC_PORT: str = "<ADD GRPC PORT HERE>"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"
grpc_cert: str = "<ADD CERTIFICATE HERE>"


options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]

# channel = grpc.secure_channel(CHANNEL, credentials, options=options)

config: ClientConfig = ClientConfig(
  host=GRPC_HOST,
  port=GRPC_PORT, 
  grpc_cert=grpc_cert)
    
print(config)
    
client: Client = Client(config=config, use_secure_channel=True)

## List all existing speech to text pipelines
All relevant configurations of the speech to text server are defined in a speech to text pipeline. 
A running server can store several such configs at the same time, such that the client can chose which one to 
pick when he sends a request to transcribe an audio file or stream.

The example below shows how to list all available pipelines by calling the **s2t_service.list_s2t_pipelines()** function, which takes a **ListS2tPipelinesRequest()** as an argument.


In [ ]:
s2t_service: Speech2Text = client.services.speech_to_text
pipelines = s2t_service.list_s2t_pipelines(request=ListS2tPipelinesRequest()).pipeline_configs
for pipeline in pipelines:
    print(pipeline)

In [ ]:
import wave 
with wave.open("audiofiles/sample_1.wav") as w:
    audio: bytes = w.readframes(w.getnframes())

In [ ]:
from typing import List, Optional

def find_pipeline_for_language(pipelines: List[Speech2TextConfig], language: str) -> Optional[Speech2TextConfig]:
    """ 
    Returns the first speech to text pipeline for the requested language. 
    If no pipline is found, return None.
    """
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

## Send a transcribe file request to the server
In general, are two different endpoints for audio transcriptions:
1. **Transcribe an audio file** 
2. **Transcribe an audio stream**

In this example, we create a **TranscribeFileRequest**, including the audio file (as bytes) and a **TranscribeRequestConfig** message, including the speech to text pipeline id, as well as optional additional parameters.
The request message is then used as an argument to the **s2t_service.transcribe_file()**, which calls the corresponding endpoint.

In [ ]:
pipeline: Speech2TextConfig = pipelines[1]

request = speech_to_text_pb2.TranscribeFileRequest(
    audio_file=audio,
    config=speech_to_text_pb2.TranscribeRequestConfig(
        s2t_pipeline_id=pipeline.id,
        ctc_decoding=speech_to_text_pb2.CTCDecoding.BEAM_SEARCH_WITH_LM,
    )
)
# Send transcription request and get response
transcribe_response = s2t_service.transcribe_file(request=request)

for transcription_message in transcribe_response.transcriptions:
    print(f"File transcript: {transcription_message.transcription}")